In [63]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import json
import re
from datetime import datetime

from sklearn.cluster import KMeans
from sklearn.feature_extraction import DictVectorizer

import nltk

In [26]:
np.random.seed(42)

In [3]:
grep_df = pd.read_csv('/root/data/lexical/grep_findings_0_499.csv')

In [5]:
snippets = grep_df['text'][:1000]
snippets

0         var CacheLineSize uintptr = CacheLinePadSize\n
1      \toffsetX86HasSSE2   = unsafe.Offsetof(cpu.X86...
2      \toffsetX86HasSSE42  = unsafe.Offsetof(cpu.X86...
3      \toffsetX86HasAVX2   = unsafe.Offsetof(cpu.X86...
4      \toffsetX86HasPOPCNT = unsafe.Offsetof(cpu.X86...
                             ...                        
995                \t\t\t\t*(*unsafe.Pointer)(k) = nil\n
996    \t\t\te := add(unsafe.Pointer(b), dataOffset+b...
997    \t\t\te := add(unsafe.Pointer(b), dataOffset+b...
998    \t\t\te := add(unsafe.Pointer(b), dataOffset+b...
999                \t\t\t\t*(*unsafe.Pointer)(e) = nil\n
Name: text, Length: 1000, dtype: object

## Extract Features

In [86]:
token_fd = nltk.FreqDist(set.union(*[tokenize(snippet) for snippet in snippets]))
all_tokens = [token for token, _ in token_fd.most_common(50)]

In [66]:
def tokenize(snippet):
    return set(list(re.split('[\s\(\)]+', snippet)))

def extract_features(snippet, all_tokens):
    tokens = tokenize(snippet)
    return {"contains_{}".format(token): token in tokens for token in all_tokens}

In [87]:
data = [extract_features(snippet, all_tokens) for snippet in snippets]

In [88]:
all_tokens

['',
 'h',
 'dataOffset+inserti*8',
 'buckets',
 '_cgo_mmap',
 'integer,',
 'datap.edata',
 'err',
 'selectnbrecv',
 'int',
 'mallocgc',
 'sp.str,',
 'base',
 'named',
 'ret',
 'dumpobj',
 '*arraytype',
 'strhash',
 'reflect_chansend',
 'selectnbsend',
 'frame',
 '&bv,',
 'oldbucket+newbit',
 'allocs,',
 'may',
 'cpuprof.extra[i]',
 'local_scan',
 'mapaccessK',
 'memhash',
 'memEnd',
 'uintptr',
 'bucketMask',
 'dumpint',
 "uintptr's",
 'new',
 'gp._panic',
 'handles',
 'dataOffset+i*8',
 'chanLock',
 'stack',
 'Loadp',
 'inheap',
 'uint64',
 'd',
 'old',
 'return',
 'minLegalPointer',
 'datap.bss',
 'or',
 'callCgoMmap']

## K-Means Clustering

In [89]:
v = DictVectorizer(sparse=False)

X = v.fit_transform(data)

In [90]:
cls = KMeans(init='k-means++', n_clusters=8).fit(X)

In [112]:
def vector_to_string(tokens):
    for key, value in tokens.items():
        if value > 0.001:
            print(key[len("contains_"):])

In [116]:
vector_to_string(v.inverse_transform(cls.cluster_centers_)[3])


new
old
return
uintptr
